# Error and Efficiency Analysis of Numerical Quadrature Methods for 
# *Upper bound Function*

## Program Description 

This program analyzes the efficiency and accuracy of various numerical quadrature methods, specifically Gauss-Legendre, Gauss-Laguerre, and hp-adaptive Gauss quadrature. It computes error estimates, determines key constants such as $\kappa$, $C^\text{hp-Gauss}$, $C^\text{GLag}$ $\alpha$, $b$ and $\gamma$ and visualizes convergence behavior. Techniques like Newton's method and analytical bounds are used for constant evaluation. The main goal is to assess how computational effort (e.g. number of nodes) relates to desired accuracy and to compare the performance of these methods across different integrands.

## Setup and helper Functions

Imports & Setup

In [ ]:
# --- Standard Library Imports ---
import math
import cmath
import time
import collections
import mpmath

# --- Third-Party Library Imports ---
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import quad
from scipy.special import kv as besselk, roots_laguerre

from mpmath import mp, mpf, log, fabs, exp, fsum, nstr, sqrt, ceil


Helper Functions

In [ ]:
def complex_quad(func, a, b, **kwargs):
    real_integral, _ = quad(lambda x: np.real(func(x)), a, b, **kwargs)
    imag_integral, _ = quad(lambda x: np.imag(func(x)), a, b, **kwargs)
    return real_integral + 1j * imag_integral

def replace_inf_with_mean(arr):
    """
    Replaces all 'inf' values in a NumPy array with the mean of the other finite values.

    Parameters:
    -----------
    arr : np.ndarray
        Input array that may contain 'inf' values.

    Returns:
    --------
    np.ndarray
        Array with 'inf' values replaced by the mean of the other finite values.
    """
    finite_values = arr[np.isfinite(arr)]  # Extract all finite values
    mean_value = np.mean(finite_values) if finite_values.size > 0 else 0  # Compute mean, or use 0 if empty
    arr[np.isinf(arr)] = mean_value  # Replace 'inf' values with the mean
    return arr

def filter_until_threshold(array, threshold=1e-14):
    """
    Filters elements from the input array until an element below the threshold is encountered.

    Parameters:
    -----------
    array : iterable
        Input array or list of numerical values.
    threshold : float, optional
        The stopping condition; elements below this value will not be included (default: 1e-14).

    Returns:
    --------
    filtered_list : list
        List of elements up to the first element below the threshold.
    count : int
        Number of elements in the filtered list.
    """
    filtered_list = [element for element in array if element >= threshold]

    return filtered_list, len(filtered_list)
def inverse_3x3(A, l):
    """
    Computes the inverse of a 3x3 matrix using the adjugate method.

    Parameters:
    -----------
    A : numpy.ndarray
        A 3x3 NumPy array representing the matrix to be inverted.
    l : int or float
        Parameter used for error reporting if the matrix is singular.

    Returns:
    --------
    Inv_A : numpy.ndarray
        The inverse of the input matrix A.

    Raises:
    -------
    ValueError:
        If the determinant is zero, indicating that the matrix is singular.
    """

    # Compute the determinant of A
    det_A = (
        A[0, 0] * (A[1, 1] * A[2, 2] - A[1, 2] * A[2, 1])
        - A[0, 1] * (A[1, 0] * A[2, 2] - A[1, 2] * A[2, 0])
        + A[0, 2] * (A[1, 0] * A[2, 1] - A[1, 1] * A[2, 0])
    )

    # Check if the matrix is singular
    if det_A == 0:
        raise ValueError(f"The Jacobian matrix is singular. Newton's method cannot be applied for l = {l}.")

    # Compute the adjugate (cofactor) matrix
    adjoint_A = np.array([
        [ A[1, 1] * A[2, 2] - A[1, 2] * A[2, 1], -(A[0, 1] * A[2, 2] - A[0, 2] * A[2, 1]),  A[0, 1] * A[1, 2] - A[0, 2] * A[1, 1]],
        [-(A[1, 0] * A[2, 2] - A[1, 2] * A[2, 0]),  A[0, 0] * A[2, 2] - A[0, 2] * A[2, 0], -(A[0, 0] * A[1, 2] - A[0, 2] * A[1, 0])],
        [ A[1, 0] * A[2, 1] - A[1, 1] * A[2, 0], -(A[0, 0] * A[2, 1] - A[0, 1] * A[2, 0]),  A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]]
    ])

    # Compute the inverse matrix
    Inv_A = adjoint_A / det_A

    return Inv_A

## Quadrature Methods

In [ ]:
theta = 1/4

In [ ]:
def gauss_legendre_quadrature_ab(f, a, b, n):
    """
    Computes the Gauss-Legendre quadrature approximation of the integral of f over [a, b].

    Parameters:
    -----------
    f : function
        The function to be integrated.
    a : float
        The lower bound of the integration interval.
    b : float
        The upper bound of the integration interval.
    n : int
        The number of quadrature nodesThe following code calculates the quadrature nodes (support points) and weights required for Gauss-Legendre quadrature. It then applies this numerical integration technique to approximate the integral of a given function $f$ over an arbitrary interval $[a, b]$ using $n$ quadrature nodes. The Gauss-Legendre quadrature method is particularly effective for smooth functions, as it achieves high accuracy with relatively few nodes by leveraging the optimal placement of integration points.
.

    Returns:
    --------
    float
        Approximate integral of f over [a, b] using Gauss-Legendre quadrature.
    """
    # Compute Gauss-Legendre nodes (x) and weights (w)
    nodes, weights = np.polynomial.legendre.leggauss(n)

    # Transform nodes to the given interval [a, b]
    transformed_nodes = (b - a) / 2 * nodes + (b + a) / 2
    transformed_weights = (b - a) / 2 * weights

    # Compute the quadrature sum
    integral_approximation = np.sum(transformed_weights * f(transformed_nodes))

    return integral_approximation

sigma = 1/3
mu = 2

def hp_gauss_legendre_quadrature(f, sigma, mu, n):
    """
    Computes the hp-adaptive Gauss-Legendre quadrature for the function f over a geometrically
    graded mesh with a grading parameter sigma and a slope parameter mu.

    Parameters:
    -----------
    f : function
        The function to be integrated over the interval [0, 1].
    sigma : float
        The grading parameter, with 0 < sigma < 1.
    mu : float
        The slope parameter controlling the polynomial degree distribution.
    n : int
        The number of subintervals in the graded mesh.

    Returns:
    --------
    I : float
        The computed integral approximation.
    N : int
        The total number of quadrature points used.
    """
    integral = 0.0
    total_nodes = 0

    for j in range(n + 1):
        # Compute polynomial degree p_j for the current subinterval
        p_j = max(2, int((n - j) * mu) + 1)

        # Compute Gauss-Legendre quadrature for the subinterval [sigma^(j+1), sigma^j]
        integral += gauss_legendre_quadrature_ab(f, sigma ** (j + 1), sigma ** j, p_j)

        # Accumulate the total number of quadrature points
        total_nodes += p_j

    return integral, total_nodes

def gauss_laguerre_quadrature(f, n, theta):
    """
    Computes the Gauss-Laguerre quadrature approximation of the integral of f over [0, ∞).

    This method is particularly useful for integrals of the form:∫₀^∞ e^(-x) f(x) dx
    where the weight function w(x) = e^(-x) is already incorporated in the quadrature.

    Parameters:
    -----------
    f : function
        The function to be integrated.
    n : int
        The number of quadrature nodes.
    theta: float
        A constant in the interval (0,1) that determines the truncation of the quadrature rule.

    Returns:
    --------
    float
        Approximate integral of f over [0, ∞) using Gauss-Laguerre quadrature.
    """
    if not (0 < theta < 1):
        raise ValueError("Theta must be in the interval (0,1).")

    # Compute Gauss-Laguerre nodes (x) and weights (w)
    nodes, weights = roots_laguerre(n)

    truncation_threshold = 4 * theta * n
    mask = (nodes > 0) & (nodes <= truncation_threshold)
    filtered_nodes = nodes[mask]
    filtered_weights = weights[mask]

    # # Output the range of filtered nodes and weights
    # nodes_range = (np.min(filtered_nodes), np.max(filtered_nodes))
    # weights_range = (np.min(filtered_weights), np.max(filtered_weights))
    #
    # print(f"Gauss-Laguerre Nodes Range: {nodes_range[0]:.15f} to {nodes_range[1]:.15f}")
    # print(f"Gauss-Laguerre Weights Range: {weights_range[0]:.15f} to {weights_range[1]:.15f}")

    # Compute the quadrature sum
    integral_approximation = np.sum(filtered_weights * f(filtered_nodes))

    return integral_approximation


## Compute Upper Bound Function

In [ ]:
beta = 0.1
nu = 1

In [ ]:
def compute_upperbound_fct_quadrature(n, l, index):
    """
    Computes the numerical quadrature of the function psi_nu using different quadrature methods.

    Parameters:
    -----------
    n : int
        Number of quadrature nodes.
    l : int
        Scaling parameter controlling the interval transformation.
    index : int
        Determines the computation method:
        - index = 1: Computes only the Gauss-Laguerre quadrature on the entire interval (0,∞) .
        - index ≠ 1: Computes all quadrature methods and compares results.


    Returns:
    --------
    f_up : float
        The computed integral using complex quadrature.
    Q_up : float
        The computed integral using Gauss quadrature.
    N : int
        Total number of quadrature points used for the hp-Gauss part.
    f_up_vec : list of float
        Contributions of each quadrature subinterval.
    Q_f_up_vec : list of float
        Contributions from Gauss quadrature methods.
    time_result : float
        Stores the computation time required to perform the quadrature.
    """

    # Define rho and zeta
    rho = 2.0 ** (-l)

    # Gauss-Laguerre Quadrature over the entire interval (0, ∞)
    if index == 1:
        return NaN

    else:
        # ** Gauss-Legendre Quadrature **
        fct_up_I = lambda eta: np.exp(eta) * (eta + np.abs(rho)) ** (-2 * nu -2)
        f_up_I = complex_quad(fct_up_I, 0, np.abs(rho))
        start_time = time.time()
        f_up_GLeg = gauss_legendre_quadrature_ab(fct_up_I, 0, np.abs(rho), n)
        end_time = time.time()
        delta_time1 = end_time - start_time

        # ** Gauss-Laguerre Quadrature (trunted)**
        fct_up_II = lambda eta: (eta + np.abs(rho)) ** (-nu -2) * np.exp(-1)
        #f_up_II = complex_quad(fct_up_II, 1, np.inf)
        f_up_II = np.exp(rho) * mpmath.gammainc(-2, 1 + rho)
        start_time = time.time()
        # Change of variables for Gauss-Laguerre quadrature
        fct_up_II_cv = lambda y: (y + 1 +np.abs(rho)) ** (-nu -2) * np.exp(-1)
        f_up_GLag = gauss_laguerre_quadrature(lambda eta: fct_up_II_cv(eta), n, theta)
        end_time = time.time()
        delta_time2 = end_time - start_time

        # ** hp-Gauss Quadrature **
        # Change of variables to the interval [0,1]
        fct_up_II_cv = lambda y: (1 - np.abs(rho)) * np.exp(-1 * (np.abs(rho)* (1-y)+y)) *(np.abs(rho)*(2-y)+y) ** (-2* nu -2)
        f_up_III = abs(complex_quad(fct_up_II_cv, 0, 1))
        start_time = time.time()
        f_up_hp_Gauss_GLeg, N = np.abs(hp_gauss_legendre_quadrature(fct_up_II_cv, sigma, mu, n))
        end_time = time.time()
        delta_time3 = end_time - start_time

        # ** Final Integration Results **
        f_up_vec = [f_up_I, f_up_II, f_up_III]
        Q_f_up_vec = [f_up_GLeg, f_up_GLag, f_up_hp_Gauss_GLeg]

        f_up = f_up_I + f_up_II + f_up_III
        Q_f_up = f_up_GLeg + f_up_GLag + f_up_hp_Gauss_GLeg
        time_result = delta_time1 + delta_time2 + delta_time3

        return f_up, Q_f_up, N, f_up_vec, Q_f_up_vec, time_result

## Compute Constans and Error Plots

### Gauss Legendre

In [ ]:
def compute_kappa(n_values, l_values):
    """
    Computes the constant kappa for given quadrature points n_values and exponents l_values.

    Parameters:
    -----------
    n_values : array-like
        Array of quadrature nodes (number of points used in the method).
    l_values : array-like
        Array of exponent values controlling rho.

    Returns:
    --------
    kappa_list : numpy.ndarray
        Array of computed kappa values for each l in l_values.
    kappa_mean : float
        Mean of all computed kappa values (ignoring NaNs).
    """

    kappa_list = []

    # Compute kappa for each l and n
    for l in l_values:
        rho = 2.0 ** (-l)

        kappa_values_n = []

        for n in n_values:
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)

            exact_error = np.abs(psi_nu_vec[0] - Q_psi_nu_vec[0])
            kappa = (exact_error * np.abs(rho) ** (2 * nu + 1) * (1 - 0.5 * np.abs(rho)) ** (2 * nu + 2)) ** (
                        -1 / (2 * n))
            kappa_values_n.append(kappa)

        if kappa_values_n:
            kappa_list.append(np.mean(kappa_values_n))
        else:
            # Handle cases where no valid kappa was found
            kappa_list.append(np.nan)

    kappa_list = np.array(kappa_list)

    # Compute the overall mean of kappa values (ignoring NaN values)
    kappa_mean = np.nanmean(kappa_list)

    # Print results
    print("\n=== Computed Constant kappa ===")
    print(f"Total computed kappa values: {len(kappa_list)}")
    print(f"Mean kappa value: {kappa_mean:.4f}")

    return kappa_list, kappa_mean
    
def GLeg_quadrature_error_plot(n_values, l_values, kappa):
    """
    Computes and plots the quadrature error and its upper bound approximation
    for given values of l and n.

    Parameters:
    -----------
    n_values : array-like
        Range of quadrature points.
    l_values : array-like
        List of exponent values l for the integral.
    kappa : array-like
        List of kappa parameters used in the upper bound approximation.
    """
    # Define the approximation function
    approx = lambda rho, n, kappa: kappa ** (-2 * n) * np.abs(rho) ** (-2 * nu - 1) * (1 - 0.5 * np.abs(rho)) ** (
                -2 * nu - 2)

    # Create subplots
    fig, axs = plt.subplots(len(l_values), 1, figsize=(12,12))

    for i, l in enumerate(l_values):
        rho = 2.0 ** (-l)

        n_list, exact_error, approx_values = [], [], []

        for n in n_values:
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            abs_error = np.abs(Q_psi_nu_vec[0] - psi_nu_vec[0])

            # Consider only errors above numerical threshold
            if abs_error >= 10 ** (-14):
                n_list.append(n)
                exact_error.append(abs_error)
                approx_values.append(approx(rho, n, kappa[i]))

        # Convert to NumPy arrays for plotting
        exact_error = np.array(exact_error)
        approx_values = np.array(approx_values)

        # Plot the results
        axs[i].semilogy(n_list, exact_error, label="Exact Quadrature Error")
        axs[i].semilogy(n_list, approx_values, label=fr"Upper Bound ($\kappa = {kappa[i] :.3f}$)")
        axs[i].grid(True, linestyle='--', color='gray', linewidth=0.1)

        axs[i].set_title(f"Semilogarithmic Plot of Remainder Term for l={l}", fontsize=17)
        axs[i].set_xlabel(r'Number of Quadrature Nodes', fontsize=12)
        axs[i].set_ylabel("Absolute Error")
        axs[i].legend()

        plt.subplots_adjust(hspace=0.2, wspace=0.3)
        for ax in axs.flat:
            ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    # Adjust layout and display plot
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()



### $hp$ - Gauss Legendre

In [ ]:
def compute_C_b_hp_gauss(n_values, l_values, gamma):
    """
    Computes the constants C and b for the upper bound function.

    Parameters:
    -----------
    n_values : array-like
        Array of quadrature nodes (number of points used in the method).
    l_values : array-like
        Array of exponent values controlling rho.
    gamma : float, optional
        Exponent parameter (default is 2).

    Returns:
    --------
    C_list : list
        List of computed C values for each l.
    b_list : list
        List of computed b values for each l.
    C_mean : float
        Mean value of C across computations.
    b_mean : float
        Mean value of b across computations.
    """

    C_list, b_list, rho_list = [], [], []

    A = lambda rho: (1 - np.abs(rho)) ** (-2 * nu - 1) * (2 / (1 + np.abs(rho))) ** (2 * nu + 2)

    # Increase l by 1 to achieve a more stable error reduction.
    # Smaller values of rho = 2^(-l) improve numerical approximation
    # and lead to a more accurate computation of the constants C and b.
    l_values = [l + 1 for l in l_values]

    for l in l_values:
        rho = 2.0 ** (-l)
        exact_err_l, N_l = [], []
        b_rho = []

        index = 0

        psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n_values[index], l, 0)
        N_l.append(N)
        exact_err_l.append(np.abs(psi_nu_vec[2] - Q_psi_nu_vec[2]))

        for n in n_values[index:]:
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)

            if N not in N_l and np.sqrt(N) < 20:
                N_l.append(N)
                exact_err_l.append(np.abs(psi_nu_vec[2] - Q_psi_nu_vec[2]))

                if len(N_l) > 7:
                    err1, err2 = exact_err_l[2], exact_err_l[-1]
                    N1, N2 = N_l[2], N_l[-1]

                    if err1 != err2 and N1 != N2:
                        b = np.log(err2 / err1) / (N2 ** (1 / gamma) - N1 ** (1 / gamma))
                        b_rho.append(b)
        if b_rho:
            b_rho_mean = np.mean(np.abs(b_rho))
            b_list.append(b_rho_mean)
        else:
            b_list.append(np.nan)

        C_rho_list = [
            exact_err_l[i] / (A(rho) * np.exp(-b_rho_mean * N ** (1 / gamma)))
            for i, N in enumerate(N_l)
        ]

        if C_rho_list:
            C_list.append(np.mean(C_rho_list))
        else:
            C_list.append(np.nan)

    C_list = np.array(C_list, dtype=np.float64).ravel()
    b_list = np.array(b_list, dtype=np.float64).ravel()

    # Compute overall mean of C and  b values
    C_mean = np.nanmean(C_list)
    b_mean = np.nanmean(b_list)

    # Compute 95% confidence interval for C and b
    C_lower_bound, C_upper_bound = np.percentile(C_list, [2.5, 97.5])
    b_lower_bound, b_upper_bound = np.percentile(b_list, [2.5, 97.5])

    # Print results
    print("\n=== Computed Constants C and b ===")
    print(f"Total computed b values: {len(b_list)}")
    print(f"Total computed C values: {len(C_list)}")
    print(f"Mean C: {C_mean:.4f}, Mean b: {b_mean:.4f}")
    print(f"95% Confidence Interval for b: [{b_lower_bound:.4f}, {b_upper_bound:.4f}]")
    print(f"95% Confidence Interval for C: [{C_lower_bound:.4f}, {C_upper_bound:.4f}]")

    return C_list, b_list, C_mean, b_mean
    
def hp_GLeg_quadrature_error_plot(n_values, l_values, C2, b2):
    """
    Computes and plots the remainder term and its upper bound approximations
    for given values of l and n.

    Parameters:
    -----------
    n_values : array-like
        Range of quadrature points.
    l_values : array-like
        List of exponent values l for the integral.
    C2 : array-like
        List of C coefficients for approximation with exponent 1/2.
    b2 : float
        Exponent parameter for approximation with 1/2.
    """
    # Define approximation functions
    approx_2 = lambda C, b, N, rho: C * (1 - np.abs(rho)) ** (-2 * nu - 1) * \
                                    (2 / (1 + np.abs(rho))) ** (2 * nu + 2) * np.exp(-b * N ** (1 / 2))

    approx_smaller = lambda C, b, N, rho: C * (1 - np.abs(rho)) ** (-2 * nu - 1) * \
                                    (2 / (1 + np.abs(rho))) ** (2 * nu + 2) * np.exp(-b * N ** (1 / 1.9915))

    # Initialize arrays to store values
    exact_n = np.empty((1, len(n_values)))
    rest_approx2_n = np.empty((1, len(n_values)))
    N_values = np.empty((1, len(n_values)))

    rest_approx_smaller_n=np.empty((1, len(n_values)))

    # Create subplots
    fig, axs = plt.subplots(len(l_values), 1, figsize=(12,12))

    for i, l in enumerate(l_values):
        rho = 2 ** (-l)

        for k, n in enumerate(n_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            exact_n[0, k] = np.abs(Q_psi_nu_vec[0] - psi_nu_vec[0])
            rest_approx2_n[0, k] = approx_2(C2[i], b2[i], N, rho)
            N_values[0, k] = N

        # Plot results
        axs[i].semilogy(N_values.T ** (1 / 2), exact_n.T, label="Exact Quadrature Error")
        axs[i].semilogy(N_values.T ** (1 / 2), rest_approx2_n.T,
                            label="Upper Bound with $\\frac{1}{2}$ \nand calculated coefficients")
        axs[i].grid(True, linestyle='--', color='gray', linewidth=0.1)
        axs[i].set_title(f"Semilogarithmic Plot of Absolut Error for l={l}", fontsize=17)
        axs[i].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=12)
        axs[i].set_ylabel("Absolute Error")
        axs[i].legend()

    plt.subplots_adjust(hspace=0.2, wspace=0.3)
    for ax in axs.flat:
        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))


    # Adjust layout and display plot
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    
#------------------
    # Create subplots
    fig, axs = plt.subplots(len(l_values), 1, figsize=(12, 12))

    for i, l in enumerate(l_values):
        rho = 2 ** (-l)

        for k, n in enumerate(n_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            exact_n[0, k] = np.abs(Q_psi_nu_vec[0] - psi_nu_vec[0])
            rest_approx2_n[0, k] = approx_2(C2[i], b2[i], N, rho)
            rest_approx_smaller_n[0,k]=approx_smaller(C2[i], b2[i], N, rho)
            N_values[0, k] = N

        # Plot results
        axs[i].semilogy(N_values.T ** (1 / 2), exact_n.T, label="Exact Quadrature Error")
        axs[i].semilogy(N_values.T ** (1 / 2), rest_approx2_n.T,
                        label="Upper Bound with $\\frac{1}{2}$ \nand calculated coefficients")
        axs[i].semilogy(N_values.T ** (1 / 2), rest_approx_smaller_n.T,
                        label="Upper Bound with $\\frac{1}{1.9915}$ \nand calculated coefficients")
        axs[i].grid(True, linestyle='--', color='gray', linewidth=0.1)
        axs[i].set_title(f"Semilogarithmic Plot of Absolut Error for l={l}", fontsize=17)
        axs[i].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=12)
        axs[i].set_ylabel("Absolute Error")
        axs[i].legend()

    plt.subplots_adjust(hspace=0.2, wspace=0.3)
    for ax in axs.flat:
        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    # Adjust layout and display plot
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### Gauss Laguerre

In [ ]:
C, alpha, gamma = sp.symbols('C alpha gamma')

def newton_method(tolerance, f, x0, l):
    """
    Implements Newton's method for solving a system of nonlinear equations.

    Parameters:
    -----------
    tolerance : float
        Convergence criterion; the algorithm stops when the difference between iterations is below this value.
    f : sympy.Matrix
        The system of nonlinear equations to be solved.
    x0 : numpy.ndarray
        Initial guess for the solution.
    l : int or float
        Parameter used for error handling and reporting.

    Returns:
    --------
    x_solution : numpy.ndarray
        The solution vector obtained using Newton's method.
    iterations : int
        The number of iterations performed before convergence.

    Raises:
    -------
    ValueError:
        If the maximum number of iterations is reached without convergence.
    """

    # Compute the Jacobian matrix of f with respect to C, alpha, and gamma
    jacobian_f = f.jacobian([C, alpha, gamma])

    # Initialize iteration variables
    x_iteration = x0
    iteration = 0
    max_iterations = 100  # Limit for iterations
    difference = tolerance + 1

    while difference > tolerance and iteration < max_iterations:
        f_x_iteration = np.array(f.subs({C: x_iteration[0], alpha: x_iteration[1], gamma: x_iteration[2]}),
                                 dtype=float).flatten()

        jacobian_x_iteration = np.array(
            jacobian_f.subs({C: x_iteration[0], alpha: x_iteration[1], gamma: x_iteration[2]}), dtype=float)
        inv_jacobian_x_iteration = inverse_3x3(jacobian_x_iteration, l)

        #x_new = x_iteration - inv_jacobian_x_iteration @ f_x_iteration

        # Alternative approach using NumPy's linear solver
        x_new = x_iteration - np.linalg.solve(jacobian_x_iteration, f_x_iteration)

        difference = np.linalg.norm(x_iteration - x_new)

        x_iteration = x_new
        iteration += 1

    # Check for non-convergence
    if iteration >= max_iterations:
        raise ValueError(f"Newton's method did not converge within {max_iterations} iterations for l = {l}.")

    return x_iteration, iteration
    
def compute_constants_via_newton(n_values, l_values, tolerance=1e-12):
    """
        Computes the constants C, alpha, and gamma using Newton's method by solving a nonlinear system.

        Parameters:
        -----------
        n_values : array-like
            List of quadrature node counts.
        l_values : array-like
            List of l values for which the computation is performed.
        tolerance : float, optional
            Convergence tolerance for Newton's method (default: 1e-12).

        Returns:
        --------
        C_alpha_gamma_means : numpy.ndarray
            Array containing the mean values of C, alpha, and gamma over all computations.
        """


    # Initialize list to store computed values of (C, alpha, gamma)
    C_alpha_gamma_values_Newton = []

    # Loop over l values
    for l in l_values:
        rho = 2.0 ** (-l)

        exact_n = np.empty(len(n_values))

        # Compute remainder term values
        for k, n in enumerate(n_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            exact_n[k] = np.abs(psi_nu_vec[1] - Q_psi_nu_vec[1])

        # Filter values until threshold
        exact_n_new, n_index = filter_until_threshold(exact_n, threshold=1e-14)
        n_new = np.arange(1, n_index + 1)

        # Select three values of n for the nonlinear system
        n1, exact_n1 = n_new[1], exact_n_new[1]
        n2, exact_n2 = n_new[-3], exact_n_new[-3]
        k = math.ceil((n_index + 1) / 2)
        n3, exact_n3 = n_new[k], exact_n_new[k]

    # Define the nonlinear system using SymPy
        f = sp.Matrix([
            C * 2.1 * (2 / (1 + sp.Abs(rho))) ** (nu + 1) * sp.sqrt(2 * sp.pi) * sp.exp(
                -alpha * ((1 + sp.Abs(rho)) / 2 * n1) ** (1 / gamma)) - exact_n1,
            C * 2.1 * (2 / (1 + sp.Abs(rho))) ** (nu + 1) * sp.sqrt(2 * sp.pi) * sp.exp(
                -alpha * ((1 + sp.Abs(rho)) / 2 * n2) ** (1 / gamma)) - exact_n2,
            C * 2.1 * (2 / (1 + sp.Abs(rho))) ** (nu + 1) * sp.sqrt(2 * sp.pi) * sp.exp(
                -alpha * ((1 + sp.Abs(rho)) / 2 * n3) ** (1 / gamma)) - exact_n3
        ])

        # Initial guess for Newton's method
        x0 = np.array([0.5, 2.5, 1.8])

        # Apply Newton's method to solve for (C, alpha, gamma)
        x_Newton, num_iterations_Newton = newton_method(tolerance, f, x0, l)

        C_alpha_gamma_values_Newton.append(x_Newton)

    # Compute the mean values of C, alpha, and gamma
    C_alpha_gamma_values_Newton = np.array(C_alpha_gamma_values_Newton, dtype=np.float64)
    C_alpha_gamma_means = np.mean(C_alpha_gamma_values_Newton, axis=0)

    # Print results
    print(f"Newton: The approximated average root is: (C, alpha, gamma) = ({C_alpha_gamma_means[0]:.4f}, {C_alpha_gamma_means[1]:.4f}, {C_alpha_gamma_means[2]:.4f}) with tolerance {tolerance}")

    return C_alpha_gamma_means
    
def compute_C_alpha_glag(n_values, l_values, gamma):
    C_list, alpha_list, rho_list = [], [], []

    A = lambda rho: 2.1 * (2 / (1 + np.abs(rho)) ** (nu + 1) * np.sqrt(2 * np.pi))

    for l in l_values:
        rho = 2.0 ** (-l)
        exact_err_l, n_l = [], []
        alpha_rho = []

        index = 2

        psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n_values[index], l, 0)
        n_l.append(n_values[index])
        exact_err_l.append(np.abs(psi_nu_vec[1] - Q_psi_nu_vec[1]))

        for n in n_values[index + 1:]:
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            n_l.append(n)
            exact_err_l.append(np.abs(psi_nu_vec[1] - Q_psi_nu_vec[1]))

            err1, err2, n1, n2 = None, None, None, None
            if len(n_l) > 4:
                err1, err2 = exact_err_l[2], exact_err_l[-1]
                n1, n2 = n_l[2], n_l[-1]

            if err1 is not None and err2 is not None and n1 is not None and n2 is not None:
                if err1 != err2 and n1 != n2:
                    numerator = log(err2) - log(err1)
                    factor = (1 + fabs(rho)) / 2
                    denominator = (factor * n1) ** (mpf(1) / gamma) - (factor * n2) ** (mpf(1) / gamma)
                    alpha_rho_val = numerator / denominator
                    alpha_rho = [alpha_rho_val]

        # Compute mean alpha_rho value
        alpha_rho_mean = mp.nan if not alpha_rho else fsum([fabs(x) for x in alpha_rho]) / len(alpha_rho)
        alpha_list.append(alpha_rho_mean)

        # C_rho_list berechnen
        C_rho_list = []
        for i, n in enumerate(n_l):
            base = (1 + fabs(rho)) / 2 * n
            exponent = base ** (mpf(1) / gamma)
            denominator = A(rho) * exp(-alpha_rho_mean * exponent)
            C_val = exact_err_l[i] / denominator
            C_rho_list.append(C_val)

        if C_rho_list:
            C_mean = fsum(C_rho_list) / len(C_rho_list)
            C_list.append(C_mean)
        else:
            C_list.append(mp.nan)

    # In mpf-Arrays umwandeln
    C_list = [mpf(x) for x in C_list]
    alpha_list = [mpf(x) for x in alpha_list]

    C_mean = fsum([x for x in C_list if not mp.isnan(x)]) / len([x for x in C_list if not mp.isnan(x)])
    alpha_mean = fsum([x for x in alpha_list if not mp.isnan(x)]) / len([x for x in alpha_list if not mp.isnan(x)])

    # Percentile-Funktion für mpmath (95% Konfidenzintervall)
    def percentile(data, perc):
        sorted_data = sorted([x for x in data if not mp.isnan(x)])
        if not sorted_data:
            return mp.nan
        k = (len(sorted_data) - 1) * perc / 100
        f = int(k)
        c = min(f + 1, len(sorted_data) - 1)
        return sorted_data[f] + (sorted_data[c] - sorted_data[f]) * (k - f)

    C_lower_bound = percentile(C_list, 2.5)
    C_upper_bound = percentile(C_list, 97.5)
    alpha_lower_bound = percentile(alpha_list, 2.5)
    alpha_upper_bound = percentile(alpha_list, 97.5)

    # Print results
    print("\n=== Computed Constants C and b ===")
    print(f"Total computed b values: {len(alpha_list)}")
    print(f"Total computed C values: {len(C_list)}")
    print(f"Mean C: {nstr(C_mean, 6)}, Mean b: {nstr(alpha_mean, 6)}")
    print(f"95% Confidence Interval for b: [{nstr(alpha_lower_bound, 6)}, {nstr(alpha_upper_bound, 6)}]")
    print(f"95% Confidence Interval for C: [{nstr(C_lower_bound, 6)}, {nstr(C_upper_bound, 6)}]")
    return C_list, alpha_list, C_mean, alpha_mean
    
def GLag_quadrature_error_plot(n_values, l_values, C, alpha, gamma):
    """
    Plots the quadrature error and its upper bound for different values of l and n.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        Maximum exponent of rho for the integral.
    C : float
        Constant parameter used in the upper bound approximation.
    alpha : list of float
        List of alpha values for different l.
    gamma : float
        Exponent parameter in the upper bound approximation.
        """
    # Define upper bound function
    r_star = lambda n, rho, gamma: ((1+ np.abs(rho))/2 * n) ** (1/gamma)
    approx_error = lambda  n, rho, alpha: C * 2.1* (2 / (1 + np.abs(rho)))**(nu +1) * np.sqrt(2 * np.pi)* np.exp(- alpha * r_star(n,rho, gamma))

    approx_error2 = lambda n, rho, alpha: C * 2.1 * (2 / (1 + np.abs(rho))) ** (nu + 1) * np.sqrt(2 * np.pi) * np.exp(
        - alpha * r_star(n, rho, 2))

    #gamma = math.ceil(test[2] * 10) / 10
    print(f" für plot gamma = {gamma}")

    fig, axs = plt.subplots(len(l_values), 1, figsize=(12,12))

    for i, l in enumerate(l_values):
        rho = 2.0 ** (-l)
        exact_n = np.empty(len(n_values))

        for k, n in enumerate(n_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            exact_n[k] = np.abs(Q_psi_nu_vec[1] - psi_nu_vec[1])

        # Filter values until threshold
        exact_n_new, n_index = filter_until_threshold(exact_n)
        n_new = np.arange(1, n_index + 1)

        # Plot
        axs[i].semilogy(n_new ** (1 / 2), exact_n_new, label="Exact Quadrature Error")
        axs[i].semilogy(n_new ** (1 / 2), approx_error2(n_new, rho, alpha[i]),
                        label="Upper Bound with $\\frac{1}{2}$ \nand calculated coefficients")
        axs[i].grid(True, linestyle='--', color='gray', linewidth=0.1)
        axs[i].set_title(f"Semilogarithmic Plot of Remainder Term for l={l}", fontsize=17)
        axs[i].set_xlabel(fr'Number of Quadrature Nodes$^{{1/2}}$', fontsize=12)
        axs[i].set_ylabel("Absolute Error")
        axs[i].legend()

        plt.subplots_adjust(hspace=0.2, wspace=0.3)

        # Adjust layout and labels
    for ax in axs.flat:
        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

#------------------
    fig, axs = plt.subplots(len(l_values), 1, figsize=(12, 12))

    for i, l in enumerate(l_values):
        rho = 2.0 ** (-l)
        exact_n = np.empty(len(n_values))

        for k, n in enumerate(n_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_upperbound_fct_quadrature(n, l, 0)
            exact_n[k] = np.abs(Q_psi_nu_vec[1] - psi_nu_vec[1])

        # Filter values until threshold
        exact_n_new, n_index = filter_until_threshold(exact_n)
        n_new = np.arange(1, n_index + 1)

        # Plot
        axs[i].semilogy(n_new ** (1 / gamma), exact_n_new, label="Exact Quadrature Error")
        axs[i].semilogy(n_new ** (1 / gamma), approx_error2(n_new, rho, alpha[i]),
                        label="Upper Bound with $\\frac{1}{2}$ \nand calculated coefficients")
        axs[i].semilogy(n_new ** (1 / gamma), approx_error(n_new, rho, alpha[i]),
                        label=f"Upper Bound with $\\gamma = {gamma:.2f}$ \nand calculated coefficients")
        axs[i].grid(True, linestyle='--', color='gray', linewidth=0.1)
        axs[i].set_title(f"Semilogarithmic Plot of Remainder Term for l={l}", fontsize=17)
        axs[i].set_xlabel(fr'Number of Quadrature Nodes$^{{1/{gamma:.2f}}}$', fontsize=12)
        axs[i].set_ylabel("Absolute Error")
        axs[i].legend()

        plt.subplots_adjust(hspace=0.2, wspace=0.3)

        # Adjust layout and labels
    for ax in axs.flat:
        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

## Cost Comparison

### Gauss-Legendre

In [ ]:
def plot_comparison_effort_gauss_legendre(n_values, l_values, exponent_max, kappa_list, compute_psi_nu_quadrature):
    """
    Computes and plots the theoretical and practical effort for Gauss-Legendre quadrature.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        List of exponent values l.
    exponent_max : int
        Maximum exponent for the error tolerance.
    kappa_list : array-like
        List of kappa values for different l.
    compute_psi_nu_quadrature : function
        Function to compute psi_nu and quadrature values.
    """
    # Define epsilon function and theoretical node estimation
    epsilon = lambda x: 10 ** (-x)
    nodes_number_GLeg = lambda rho, epsilon, kappa: (1 / (2 * np.log(kappa))) * (
        - (2 * nu + 1) * np.log(np.abs(rho))
        - (2 * nu + 2) * np.log(1 - 0.5 * np.abs(rho))
        - np.log(epsilon)
    )

    # Initialize storage matrices
    n_GLeg_values = np.zeros((exponent_max, len(l_values)))
    for i in range(0, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = 2.0 ** (-l)
            eps_value = epsilon(i)
            n_GLeg_values[i - 1, j] = np.ceil(nodes_number_GLeg(rho, eps_value, kappa_list[j]))

    Y_epsilon = np.array([epsilon(x) for x in range(0, exponent_max)])
    absolut_error_GLeg = np.zeros((len(l_values), len(n_values)))

    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            # Compute absolute error
            absolut_error_GLeg[j, i] = np.abs(np.abs(psi_nu_vec[0]) - np.abs(Q_psi_nu_vec[0]))

    print("Shape of n_GLeg_values[:, j]:", n_GLeg_values[:, j].shape)
    print("Shape of Y_epsilon:", Y_epsilon.shape)

    # Plot results
    length = math.ceil(len(l_values) / 2)
    fig, axs = plt.subplots(2, length, figsize = (18.5, 8.9))

    if length == 1:
        axs = axs.flatten()

    plt.suptitle("Theoretical and Practical Effort for Node Points Gauss-Legendre", fontsize=15)

    for j, l in enumerate(l_values[:length]):
        axs[0, j].semilogy(n_values, absolut_error_GLeg[j, :], label=f"Practical")
        axs[0, j].semilogy(n_GLeg_values[:, j], Y_epsilon, marker='o', linestyle='-', label=f"Theoretical")
        #axs[0, j].legend()
        axs[0, j].set_title(f"l = {l}")
        axs[0, j].set_xlabel(r'Number of Quadrature Nodes', fontsize=8)
        #axs[0, j].set_ylabel("Absolute Error")
        axs[0, j].set_ylim(None, 10 ** 0 + 1)
    axs[0, 0].set_ylabel("Absolute Error")

    for j, l in enumerate(l_values[length:]):
        axs[1, j].semilogy(n_values, absolut_error_GLeg[length + j, :], label=f"Practical")
        axs[1, j].semilogy(n_GLeg_values[:, length + j], Y_epsilon, marker='o', linestyle='-', label=f"Theoretical")
        #axs[1, j].legend()
        axs[1, j].set_title(f"l = {l}")
        axs[1, j].set_xlabel(r'Number of Quadrature Nodes', fontsize=8)
        #axs[1, j].set_ylabel("Absolute Error")
        axs[1, j].set_ylim(None, 10 ** 0 + 1)
    axs[1, 0].set_ylabel("Absolute Error")

    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    legend = fig.legend(["Practical", "Theoretical"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2, fontsize=10)
     # Speichern des Plots mit hoher Qualität

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### $hp$-Gauss-Legendre

In [ ]:
def plot_comparison_effort_hp_gauss(n_values, l_values, exponent_max, C_list, b_list, compute_psi_nu_quadrature):
    """
    Computes and plots the theoretical and practical effort for hp-Gauss-Laguerre quadrature.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        List of exponent values l.
    exponent_max : int
        Maximum exponent for the error tolerance.
    C_list : array-like
        List of C values for different l.
    b_list : array-like
        List of b values for different l.
    compute_psi_nu_quadrature : function
        Function to compute psi_nu and quadrature values.
    """
    # Define epsilon function and theoretical node estimation
    epsilon = lambda x: 10 ** (-x)
    nodes_number_hp_Gauss = lambda rho, epsilon, C, b: (
        (np.log(C) + (-2 * nu - 1) * np.log(1 - np.abs(rho))
        + (2 * nu + 2) * np.log(2 / (1 + np.abs(rho))) - np.log(epsilon)) / b
    ) ** 2

    # Initialize storage matrices
    n_hp_Gauss_values = np.zeros((exponent_max, len(l_values)))
    for i in range(0, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = 2.0 ** (-l)
            eps_value = epsilon(i)
            n_hp_Gauss_values[i - 1, j] = np.ceil(nodes_number_hp_Gauss(rho, eps_value, C_list[j], b_list[j]))

    Y_epsilon = np.array([epsilon(x) for x in range(0, exponent_max)])
    num_iterations = np.zeros((len(l_values), len(n_values)))
    absolut_error_hp_Gauss = np.zeros((len(l_values), len(n_values)))

    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            # Compute psi_nu and quadrature approximations
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            # Compute absolute error
            num_iterations[j,i] = N
            absolut_error_hp_Gauss[j, i] = np.abs(np.abs(psi_nu_vec[2]) - np.abs(Q_psi_nu_vec[2]))


    # Plot results
    length = math.ceil(len(l_values) / 2)
    fig, axs = plt.subplots(2, length, figsize=(18.5, 8.9))

    if length == 1:
        axs = axs.flatten()

    plt.suptitle(r'Theoretical and Practical Effort for Node Points $hp$-Gauss', fontsize=15)

    for j, l in enumerate(l_values[:length]):
        axs[0, j].semilogy(num_iterations[j, :] ** (1/2), absolut_error_hp_Gauss[j, :], label=f"Practical l={l}")
        axs[0, j].semilogy(n_hp_Gauss_values[:, j] ** (1 / 2), Y_epsilon, marker='o', linestyle='-', label=f"Theoretical l={l}")
        #axs[0, j].legend()
        axs[0, j].set_title(f"l = {l}")
        axs[0, j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        #axs[0, j].set_ylabel("Absolute Error")
        axs[0, j].set_ylim(None, 10 ** 0 + 1)
    axs[0, 0].set_ylabel("Absolute Error")

    for j, l in enumerate(l_values[length:]):
        axs[1, j].semilogy(num_iterations[length+j, :] ** (1/2), absolut_error_hp_Gauss[length+j, :], label=f"Practical l={l}")
        axs[1, j].semilogy(n_hp_Gauss_values[:,length+ j] ** (1 / 2), Y_epsilon, marker='o', linestyle='-', label=f"Theoretical l={l}")
        #axs[1, j].legend()
        axs[1, j].set_title(f"l = {l}")
        axs[1, j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        #axs[1, j].set_ylabel("Absolute Error")
        axs[1, j].set_ylim(None, 10 ** 0 + 1)

    axs[1, 0].set_ylabel("Absolute Error")
    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    legend = fig.legend(["Practical", "Theoretical"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2,
                        fontsize=10)
    # Speichern des Plots mit hoher Qualität

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### Gauss-Laguerre

In [ ]:
def plot_comparison_effor_gauss_laguerre(n_values, l_values, exponent_max, C_list, alpha_list, C_tilde_mean, gamma, compute_psi_nu_quadrature):
    """
    Computes and plots the theoretical and practical effort for Gauss-Laguerre quadrature.

    Parameters:
    -----------
    n_values : array-like
        Number of quadrature points.
    l_values : array-like
        List of exponent values l.
    exponent_max : int
        Maximum exponent for the error tolerance.
    C_list : array-like
        List of C values for different l.
    alpha_list : array-like
        List of alpha values for different l.
    gamma : int
        Exponent for the n.
    compute_psi_nu_quadrature : function
        Function to compute psi_nu and quadrature values.
    """
    # Define epsilon function and theoretical node estimation
    epsilon = lambda x: mpf(10) ** (-x)

    # Umgerechnete Formel zur Knotenzahl (nur mpmath-Funktionen!)
    def nodes_number_GLag(rho, epsilon, C, alpha, gamma):
        factor = 2 / (1 + fabs(rho))
        inner = (-1 / alpha) * log(epsilon / (C * mpf('2.1') * factor ** (nu + 1) * sqrt(2 * mp.pi)))
        return factor * (inner ** gamma)

    # Initialisierung
    n_GLag_values = [[mp.nan for _ in l_values] for _ in range(exponent_max)]

    for i in range(0, exponent_max + 1):
        for j, l in enumerate(l_values):
            rho = mpf(2) ** (-l)
            eps_value = epsilon(i)
            n_value = nodes_number_GLag(rho, eps_value, C_list[j], alpha_list[j], gamma)
            n_GLag_values[i - 1][j] = ceil(n_value)

    #Epsilon-Werte (für Y-Achse im Plot)
    Y_epsilon = [epsilon(x) for x in range(0, exponent_max)]

    # Initialisierung absolute Fehler
    absolut_error_GLag = [[mp.nan for _ in n_values] for _ in l_values]

    for i, n in enumerate(n_values):
        for j, l in enumerate(l_values):
            psi_nu, Q_psi_nu, N, psi_nu_vec, Q_psi_nu_vec, time_result = compute_psi_nu_quadrature(n, l, 0)
            abs_diff = fabs(fabs(psi_nu_vec[1]) - fabs(Q_psi_nu_vec[1]))
            absolut_error_GLag[j][i] = abs_diff

    # Plot results
    length = math.ceil(len(l_values) / 2)
    fig, axs = plt.subplots(2, length, figsize=(18.5, 8.9))

    if length == 1:
        axs = axs.flatten()

    plt.suptitle(r'Theoretical and Practical Effort for Node Points Gauss-Laguerre', fontsize=15)

    for j, l in enumerate(l_values[:length]):
        axs[0][j].semilogy(n_values ** 0.5, absolut_error_GLag[j], label=f"Practical l={l}")

        x_vals = [float(n_GLag_values[i][j]) ** 0.5 for i in range(len(n_GLag_values))]
        y_vals = [float(e) for e in Y_epsilon]
        axs[0][j].semilogy(x_vals, y_vals, marker='o', linestyle='-', label=f"Theoretical l={l}")

        #axs[0, j].legend()
        axs[0][j].set_title(f"l = {l}")
        axs[0][j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        #axs[0, j].set_ylabel("Absolute Error")
        axs[0][j].set_ylim(None, 10 ** 0 + 1)
    axs[0][0].set_ylabel("Absolute Error")

    for j, l in enumerate(l_values[length:]):
        y_vals = [float(val) for val in absolut_error_GLag[length + j]]
        x_vals = [float(n) ** 0.5 for n in n_values]
        axs[1][j].semilogy(x_vals, y_vals, label=f"Practical l={l}")

        x_vals = [float(n_GLag_values[i][length + j]) ** 0.5 for i in range(len(n_GLag_values))]
        y_vals = [float(e) for e in Y_epsilon]
        axs[1][j].semilogy(x_vals, y_vals, marker='o', linestyle='-', label=f"Theoretical l={l}")

        #axs[1, j].legend()
        axs[1][ j].set_title(f"l = {l}")
        axs[1][ j].set_xlabel(r'Number of Quadrature Nodes$^{1/2}$', fontsize=8)
        #axs[1, j].set_ylabel("Absolute Error")
        axs[1][ j].set_ylim(None, 10 ** 0 + 1)
    axs[1][ 0].set_ylabel("Absolute Error")

    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    legend = fig.legend(["Practical", "Theoretical"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=2,
                        fontsize=10)
    # Speichern des Plots mit hoher Qualität

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

## Experiment Setup

Define quadrature parameters

In [ ]:

n_values_GLeg = np.arange(2,20)
n_values_hpGLeg = np.arange(2,20)
n_values_GLag = np.arange(2,40)
l_values = np.arange(1,11)
l_values_plots = [2, 10]

exponent_max = 12

Calculation of Constants and Comparison

In [ ]:
print(f"------------GLeg-------------------")
kappa_list, kappa_mean = compute_kappa(n_values_GLeg, l_values)
print(f"kappa = {[f'{kappa:.3f}' for kappa in kappa_list]}")
kappa_list = replace_inf_with_mean(kappa_list)
kappa_list = [x + 0.2 for x in kappa_list]
print(f"replaced kappa = {[f'{kappa:.3f}' for kappa in kappa_list]}")


kappa_list_plot = [kappa_list[i-1] for i in l_values_plots]
GLeg_quadrature_error_plot(n_values_GLeg, l_values_plots, kappa_list_plot)

plot_comparison_effort_gauss_legendre(n_values_GLeg, l_values, exponent_max, kappa_list, compute_upperbound_fct_quadrature)

print(f"------------hp-GLeg-------------------")
C_list, b_list, C_mean, b_mean = compute_C_b_hp_gauss(n_values_hpGLeg, l_values , 2)
C_list = [x for x in C_list]
b_list = [x+0.1 for x in b_list]
# b_list[-3]= 1.15
# b_list[-2] = b_list[-1]= 1.15
print(f"C = {[f'{c:.3f}' for c in C_list]}")
print(f"b = {[f'{b:.3f}' for b in b_list]}")


C_list_plot = [C_list[i-1] for i in l_values_plots]
b_list_plot = [b_list[i-1] for i in l_values_plots]
hp_GLeg_quadrature_error_plot(n_values_hpGLeg, l_values_plots, C_list_plot, b_list_plot)
plot_comparison_effort_hp_gauss(n_values_hpGLeg, l_values, exponent_max, C_list, b_list, compute_upperbound_fct_quadrature)



print(f"------------GLag-------------------")
test = compute_constants_via_newton(n_values_GLag, l_values)
if not np.isnan(test[2]):  # Falls kein NaN
    gamma = test[2]
else:
    print("Warning: gamma is NaN, using default value 1.8")
    gamma = 1.7626


C_tilde_list, alpha_list, C_tilde_mean, alpha_mean = compute_C_alpha_glag(n_values_GLag, l_values, gamma)
print(f"gamma = {float(gamma):.3f}")
print(f"C = {[f'{float(c):.3f}' for c in C_tilde_list]}")
print(f"alpha = {[f'{float(alpha):.3f}' for alpha in alpha_list]}")

C_tilde_list_plot = [float(C_tilde_list[i - 1]) for i in l_values_plots]
alpha_list_plot = [float(alpha_list[i - 1]) for i in l_values_plots]

GLag_quadrature_error_plot(n_values_GLag, l_values_plots, C_tilde_mean/2, alpha_list_plot, gamma)

plot_comparison_effor_gauss_laguerre(n_values_GLag, l_values, exponent_max, C_tilde_list, alpha_list, C_tilde_mean, gamma, compute_upperbound_fct_quadrature)
